In [5]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [6]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [34]:
##Implementations of the required function

#Linear regression using gradient descent
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    nIters = 0 #Keep count of iterations
    w = initial_w #Current weights
    n = len(y) #Number of observations
    while (nIters < max_iters):
        e = y - np.dot(tx, w) #Residual vector
        gradient = -np.dot(np.transpose(tx), e) / n #Gradient
        w -= gamma * gradient #A step towards negative gradient
        nIters += 1 #Update number of iterations
    e = y - np.dot(tx, w) #Compute final residuals
    return w, np.dot(np.transpose(e), e) / (2 * n) #Return weights and loss (2n as a scaler)

#Linear regression using stochastic gradient descent
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    nIters = 0 #Keep track of iterations
    w = initial_w #Update w
    n = len(y) #Number of datapoints
    while (nIters < max_iters):
        index = np.random.randint(0, n) #Pick a row uniformly
        row = tx[index, :] #Select the chosen row 
        e = y[index] - np.dot(row, w) #Calculate the estimate for error
        gradient = -np.dot(np.transpose(row), e) #Calculate the estimate for the gradient
        w -= gamma * gradient #Update w
        nIters += 1 #Update number of iterations
    e = y - np.dot(tx, w) #Calculate the residuals for the final loss
    return w, np.dot(np.transpose(e), e) / (2 * n) #Return the weights and the loss (2n as a scaler)

#Least squares regression using normal equations
def least_squares(y, tx):
    xtx = np.dot(np.transpose(tx), tx) #Calculate the Gram matrix
    w = np.dot(np.dot(np.linalg.inv(xtx), np.transpose(tx)), y) #Calculate the weigths
    e = y - np.dot(tx, w) #Calculate the residuals
    loss = np.dot(np.transpose(e), e) / (2 * len(y)) #Calculate the loss (2n as a scaler)
    return w, loss

#Ridge regression using normal equations
def ridge_regression(y, tx, lambda_):
    xtx = np.dot(np.transpose(tx), tx) + lambda_ * np.identity(np.shape(tx)[1]) #Calculate the modified Gram matrix
    w = np.dot(np.dot(np.linalg.inv(xtx), np.transpose(tx)), y) #Calculate the weigths
    e = y - np.dot(tx, w) #Calculate residuals
    loss = np.dot(np.transpose(e), e) / (2 * len(y)) #Calculate the loss (2n as a scaler)
    return w, loss

#A helper for the logistic regression
def sigmoid(x):
    return np.exp(x) / (1 + np.exp(x))

#Logistic regression using gradient descent
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    nIters = 0
    w = initial_w
    n = len(y)
    while (nIters < max_iters): 
        gradient = np.dot(np.transpose(tx), sigmoid(np.dot(tx, w)) - y)
        w -= gamma * gradient #Calculate new w
        nIters += 1 #Update number of iterations
    loss = np.sum(np.log(1 + np.exp(np.dot(tx, w))) - y * np.dot(tx, w))
    return w, loss #Return weights and loss

#Regularized logistic regression using gradient descent or SGD
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    
    nIters = 0
    w = initial_w
    n = len(y)
    while (nIters < max_iters): 
        
        #calculate the hessian matrix
        S = np.zeros(n)
        for k in range(n):
            sig = sigmoid(np.dot(np.transpose(tx[k]), w))
            S[k] = sig * (1 - sig)
        
        tmp = np.zeros((n, np.shape(tx)[1]))
        for k in range(n):
            tmp[k, :] = tx[k, :] * S[k]
        hessian = np.dot(np.transpose(tx), tmp)
        
        #calculate the gradient
        gradient = np.dot(np.transpose(tx), sigmoid(np.dot(tx, w)) - y)
        
        #update w
        w = w - gamma*np.dot(np.linalg.inv(hessian),gradient)
        
        nIters += 1 #Update number of iterations
        
    #calculate the loss
    loss = 0
    for k in range(n):
        loss += np.log(1 + np.exp(np.dot(np.transpose(tx[k]),w))) - np.dot(y[k],np.dot(np.transpose(tx[k]),w))
    loss += lambda_/2* np.linalg.norm(w)
    
    return w, loss #Return weights and loss

## Tests:
#w1, loss1 = least_squares(y, tX)
#w2, loss2 = ridge_regression(y, tX, 0.02)
#w3, loss3 = least_squares_GD(y, tX, 10 * w2, 20, 0.01)
#w4, loss4 = least_squares_SGD(y, tX, np.ones(30), 10, 0.01)
#y2 = np.copy(y)
#y2[y2 == -1] = 0
#w4, loss4 = logistic_regression(y2, tX, np.ones(30), 10, 0.01)


In [10]:
## Functions for cleaning and manipulating the data
def replace_with_mean(data, replacedValue):
    means = []
    def replace_one_col(x):
        m = np.mean(x[x != replacedValue])
        x[x == replacedValue] = m
        means.append(m)
        return x
    return np.apply_along_axis(replace_one_col, 0, np.copy(data)), means

def replace_with_values(data, values, replacedValue):
    result = np.copy(data)
    for i in range(np.shape(data)[1]):
        for j in range(np.shape(data)[0]):
            if result[j, i] == replacedValue:
                result[j, i] = values[i]
    return result
            
def centralize_data(data):
    return np.apply_along_axis(lambda x: x - np.mean(x), 0, np.copy(data))

def subtract_values(data, values):
    result = np.copy(data)
    for i in range(np.shape(data)[1]):
        for j in range(np.shape(data)[0]):
            result[j, i] = result[j, i] - values[i]
    return result


def scale_data(data):
    deviations = np.apply_along_axis(lambda x: np.std(x), 0, np.copy(data))
    return np.apply_along_axis(lambda x: x / np.std(x), 0, np.copy(data)), deviations

def divide_by_values(data, values):
    result = np.copy(data)
    for i in range(np.shape(data)[1]):
        for j in range(np.shape(data)[0]):
            result[j, i] = result[j, i] / values[i]
    return result

def add_constant_term(data):
    a = np.ones((np.shape(data)[0], np.shape(data)[1] + 1))
    a[:, :-1] = data
    return a

def add_missing_value_info(data, missingValue):
    a = np.zeros((np.shape(data)[0], np.shape(data)[1] + 1))
    a[:, :-1] = data
    a[:, np.shape(a)[1] - 1] = np.apply_along_axis(lambda x: np.sum(x == missingValue), 1, data)
    return a


In [8]:
## Functions for model validation
def split_data(x, y, ratio, seed = 1):
    #Split the dataset based on the split ratio.
    np.random.seed(seed) #Set a seed for reproducilbility
    trainSize = int(np.round(len(y) * ratio)) #Calculate the desired size of the train data set 
    trainIndexes = np.random.choice(len(y), trainSize, False) #Sample the training data
    testIndexes = np.setdiff1d(np.arange(len(y)), trainIndexes, assume_unique=True) #Select the test data
    return x[trainIndexes, :], y[trainIndexes], x[testIndexes, :], y[testIndexes]


In [46]:
#Load test data
DATA_TEST_PATH = '../../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [11]:
tX1, means = replace_with_mean(tX, -999)
tX1, deviations = scale_data(centralize_data(tX1))

In [38]:
### Fit models

#We compare different models using the prediction accuracies. The comparison is done using cross validation.
#We first compare whetrher the GD, SGD and the closed least square functions give the same models. Then we fit a ridge regression model
#using different regularization parameters and finally a logistic regression model and regularized logistic regression
#models using different parameters. The best model is chosen using cross validation.


##Linear models:
#The models are fitted to the preprocessed and centralized data and a learning rate of 0.1
#is used in the GD and SGD algorithms.

#1. A model using GD
weightsGD, lossGD = least_squares_GD(y, tX1, np.zeros(np.shape(tX1)[1]), 2000, 0.1)

#2. A model using SGD
weightsSGD, lossSGD = least_squares_SGD(y, tX1, np.zeros(np.shape(tX1)[1]), 200000, 0.001)

#3. A model using closed form solution for the least squares
weightsLSQ, lossLSQ = least_squares(y, tX1)

#Check for differences in losses and weights

print("Loss ratio1: ", lossLSQ / lossGD)
print("Loss ratio2: ", lossLSQ / lossSGD)
print(weightsGD)
print(weightsSGD)
print(weightsLSQ)
print("Max weight difference1: ", np.max(weightsLSQ - weightsSGD))
print("Max weight difference1: ", np.max(weightsLSQ - weightsGD))

#The results are close to each other. In the future, only least_squares will be used.



Loss ratio1:  0.9999968428610891
Loss ratio2:  0.989583637460409
[ 9.63655762e-03 -2.54736019e-01 -2.63503443e-01 -1.16337530e-03
  2.18326773e-02  9.00606246e-02  4.84226674e-03  2.82008481e-01
 -2.81645647e-02  5.56983585e-02 -1.88136508e-01  1.18049028e-01
  7.66256122e-02  1.74657749e-01 -7.79826766e-04 -8.30897863e-04
  2.78600745e-01 -8.61053861e-04  2.51520388e-03  1.03680159e-01
  9.36298625e-04 -4.70340922e-02  4.08260971e-02 -4.81893744e-02
  6.52475974e-04  1.89116701e-04 -3.69103668e-02  1.55888050e-03
 -1.74013359e-03 -3.69016108e-02]
[ 0.0181864  -0.25927543 -0.25542027 -0.00553715  0.04847504  0.07236159
 -0.02086306  0.26175669 -0.01584657  0.04067617 -0.15919671  0.08658388
  0.08730788  0.17664583  0.00746078 -0.00470276  0.2789183  -0.00586078
 -0.02104699  0.14647441 -0.01099862 -0.01614169  0.07305421 -0.05265507
 -0.0238358   0.00175934 -0.0575094  -0.01145926  0.00138146 -0.05516273]
[ 9.63468389e-03 -2.54718967e-01 -2.63502819e-01 -1.10015735e-03
  2.18421784e-0

In [65]:
### In this part, we will fit a model using ridge regression and try to find an optimal 
#regularization parameter using 5 fold cross validation. The used split ratio is 0.9.

#The losses will be calculated using prediction accuracy with a threshold of 0,
#as the final model will be rated using this same criterion.

folds = 5
lambdas = np.logspace(-4, 0, 30)
accuracies = np.zeros((len(lambdas), folds))

np.random.seed(123)
for k in range(folds):
    i = 0
    trainX, trainY, testX, testY = split_data(tX, y, 0.9, seed = 1)
    for lambda_ in lambdas:
        weights, _ = ridge_regression(trainY, trainX, lambda_)
        predictions = np.dot(testX, weights)
        predictions[predictions < 0] = -1
        predictions[predictions >= 0] = 1
        #Use a naive loss function (prediction accuracy)
        accuracy = np.mean(predictions == testY)
        accuracies[i, k] = accuracy
        i += 1

# Select the model with the highest mean accuracy to be used later (some of the smallest values seem to be as good)
# It seems that the model performs best when the lambda is close to zero.
ridgeLambda = lambdas[np.argmax(np.mean(accuracies, axis = 1))]


In [73]:
##Next, regularized logistic regression models will be fitted using the same kind of process as with the ridge regression.
##However, as the optimization algorithm is quite slow, only a 2 fold cross validation is performed and the search space is
#smaller

folds = 2
initialW = np.zeros(np.shape(trainX)[1])
lambdas = np.logspace(-4, 0, 10)
accuracies = np.zeros((len(lambdas), folds))

np.random.seed(123)
for k in range(folds):
    i = 0
    trainX, trainY, testX, testY = split_data(tX, y, 0.9, seed = 1)
    trainX[trainX == -1] = 0
    trainY[trainY == -1] = 0
    print(k)
    
    for lambda_ in lambdas:
        print("New lambda")
        weights, _ = reg_logistic_regression(trainY, trainX, lambda_, initialW, 20, 0.1)
        predictions = np.dot(testX, weights)
        predictions[predictions < 0.5] = 0
        predictions[predictions > 0] = 1
        #Use a naive loss function (prediction accuracy)
        accuracy = np.mean(predictions == testY)
        accuracies[i, k] = accuracy
        i += 1

# Select the model with the highest mean accuracy to be used later
regLogRegLambda = lambdas[np.argmax(np.mean(accuracies, axis = 1))]


0
New lambda
New lambda


KeyboardInterrupt: 

In [75]:
##At this section, we are comparing different regression models using cross validation
##The compared models are:
#1: Linear regression model fitted by the least_squares function
#2: Ridge regression using the lambda found earlier
#3: Regularized logistic regression using the lambda found earlier
#4: A logistic regression model

folds = 5
initialW = np.zeros(np.shape(trainX)[1])
lambdas = np.logspace(-4, 0, 30)
accuracies = np.zeros((4, folds))

np.random.seed(200)
for k in range(folds):
    print(k)
    trainX, trainY, testX, testY = split_data(tX1, y, 0.9, seed = 1)
    for lambda_ in lambdas:
        
        print("Lsq")
        #Least squares linear regression
        lsqWeights, _ = least_squares(trainY, trainX)
        lsqPreds = np.dot(testX, lsqWeights)
        lsqPreds[lsqPreds < 0] = -1
        lsqPreds[lsqPreds >= 0] = 1
        accuracies[0, k] = np.mean(lsqPreds == testY)
        
        print("Ridge")
        #Penalized linear regression
        ridgeWeights, _ = ridge_regression(trainY, trainX, ridgeLambda)
        ridgePreds = np.dot(testX, ridgeWeights)
        ridgePreds[ridgePreds < 0] = -1
        ridgePreds[ridgePreds >= 0] = 1
        accuracies[1, k] = np.mean(ridgePreds == testY)
        
        print("Logistic regression")
        #Logistic regression
        trainY[trainY == -1] = 0
        testY[testY == -1] = 0
        logisticWeights, _ = logistic_regression(trainY, trainX, initialW, 20, 0.1)
        logisticPreds = sigmoid(np.dot(testX, logisticWeights))
        logisticPreds[logisticPreds > 0.5] = 1
        logisticPreds[logisticPreds <= 0.5] = 0
        accuracies[2, k] = np.mean(logisticPreds == testY)
        
        print("Regularized logistic regression")
        #Regularized logistic regression
        regLogRegWeights, _ = reg_logistic_regression(trainY, trainX, regLogRegLambda, initialW, 20, 0.1)
        logisticPreds = sigmoid(np.dot(testX, regLogRegWeights))
        logisticPreds[logisticPreds > 0.5] = 1
        logisticPreds[logisticPreds <= 0.5] = 0
        accuracies[3, k] = np.mean(logisticPreds == testY)


0
Lsq
Ridge
Logistic regression


C:\Users\Alvari.Kone\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Alvari.Kone\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in greater
C:\Users\Alvari.Kone\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in less_equal


Regularized logistic regression


C:\Users\Alvari.Kone\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in greater
C:\Users\Alvari.Kone\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in less_equal


Lsq
Ridge
Logistic regression
Regularized logistic regression


KeyboardInterrupt: 

In [ ]:
#According to the cross validation, the best model is ???
#Fit this model using the whole training data and do predictions



In [140]:
## Use the optimal weights (lambda = 0.046415888) to make predictions for the real test data
tmp = replace_with_values(add_missing_value_info(tX_test, -999), means, -999)
tmp[tmp[:, np.shape(tmp)[1] -1] > 0, np.shape(tmp)[1] -1] = 1
model2TestData = add_constant_term(divide_by_values(subtract_values(tmp, means), deviations))
testPredictions = sigmoid(np.dot(model2TestData, bestW))
testPredictions[testPredictions < 0.5] = -1
testPredictions[testPredictions > 0] = 1
#Save predictions
OUTPUT_PATH = '../../predictions/regularized_logistic_regression2.csv' 
create_csv_submission(ids_test, testPredictions, OUTPUT_PATH)